# Parte 4 - Embeddings 

In [ ]:
import fasttext

# fasttext.util.download_model('pt', if_exists='ignore')
model = fasttext.load_model("..\\cc.pt.300.bin")
vector = model.get_word_vector("jesus")
print(vector[:10]) 

In [ ]:
import pandas as pd

hinos_analise:pd.DataFrame = pd.read_pickle("..\\assets\\hinos_analise_tokens.pkl")

In [ ]:
import numpy as np


def embed_text(tokens, model):
    vectors = [model.get_word_vector(word) for word in tokens]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.get_dimension())


hinos_analise["embedding"] = hinos_analise["tokens_no_stops"].apply(
    lambda t: embed_text(t, model)
)
hinos_analise

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

sims = cosine_similarity(list(hinos_analise["embedding"]))
sims = pd.DataFrame(sims, index=hinos_analise.index, columns=hinos_analise.index)
sims

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 10))
sns.heatmap(sims, cmap="viridis", annot=False)
plt.title("Similaridade entre hinos (Word Embeddings)")
plt.show()

In [ ]:
high_similarity_word2vec = sims[
    (sims > 0.5) & (sims < 1.0)
].stack()  # .reset_index()
high_similarity_word2vec = high_similarity_word2vec[
    high_similarity_word2vec.index.get_level_values(0)
    < high_similarity_word2vec.index.get_level_values(1)
]
high_similarity_word2vec.sort_values(ascending=False)